# Zadatak klasifikacije teksta

U ovom modulu započet ćemo s jednostavnim zadatkom klasifikacije teksta temeljenim na skupu podataka **[AG_NEWS](http://www.di.unipi.it/~gulli/AG_corpus_of_news_articles.html)**: klasificirat ćemo naslove vijesti u jednu od 4 kategorije: Svijet, Sport, Poslovanje i Znanost/Tehnologija.

## Skup podataka

Za učitavanje skupa podataka koristit ćemo API **[TensorFlow Datasets](https://www.tensorflow.org/datasets)**.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds

# In this tutorial, we will be training a lot of models. In order to use GPU memory cautiously,
# we will set tensorflow option to grow GPU memory allocation when required.
physical_devices = tf.config.list_physical_devices('GPU') 
if len(physical_devices)>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

dataset = tfds.load('ag_news_subset')

Sada možemo pristupiti treninzima i testnim dijelovima skupa podataka koristeći `dataset['train']` i `dataset['test']`:


In [3]:
ds_train = dataset['train']
ds_test = dataset['test']

print(f"Length of train dataset = {len(ds_train)}")
print(f"Length of test dataset = {len(ds_test)}")

Length of train dataset = 120000
Length of test dataset = 7600


Ispišimo prvih 10 novih naslova iz našeg skupa podataka:


In [4]:
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

for i,x in zip(range(5),ds_train):
    print(f"{x['label']} ({classes[x['label']]}) -> {x['title']} {x['description']}")

3 (Sci/Tech) -> b'AMD Debuts Dual-Core Opteron Processor' b'AMD #39;s new dual-core Opteron chip is designed mainly for corporate computing applications, including databases, Web services, and financial transactions.'
1 (Sports) -> b"Wood's Suspension Upheld (Reuters)" b'Reuters - Major League Baseball\\Monday announced a decision on the appeal filed by Chicago Cubs\\pitcher Kerry Wood regarding a suspension stemming from an\\incident earlier this season.'
2 (Business) -> b'Bush reform may have blue states seeing red' b'President Bush #39;s  quot;revenue-neutral quot; tax reform needs losers to balance its winners, and people claiming the federal deduction for state and local taxes may be in administration planners #39; sights, news reports say.'
3 (Sci/Tech) -> b"'Halt science decline in schools'" b'Britain will run out of leading scientists unless science education is improved, says Professor Colin Pillinger.'
1 (Sports) -> b'Gerrard leaves practice' b'London, England (Sports Network

## Vektorizacija teksta

Sada trebamo pretvoriti tekst u **brojeve** koji se mogu predstaviti kao tenzori. Ako želimo reprezentaciju na razini riječi, trebamo napraviti dvije stvari:

* Koristiti **tokenizator** za razdvajanje teksta na **tokene**.
* Izgraditi **rječnik** tih tokena.

### Ograničavanje veličine rječnika

U primjeru s AG News skupom podataka, veličina rječnika je prilično velika, više od 100 tisuća riječi. Općenito govoreći, ne trebaju nam riječi koje se rijetko pojavljuju u tekstu — samo nekoliko rečenica će ih sadržavati, a model iz njih neće ništa naučiti. Stoga ima smisla ograničiti veličinu rječnika na manji broj tako da proslijedimo argument konstruktoru vektorizatora:

Oba ova koraka mogu se obaviti pomoću sloja **TextVectorization**. Instancirajmo objekt vektorizatora, a zatim pozovimo metodu `adapt` kako bismo prošli kroz sav tekst i izgradili rječnik:


In [5]:
vocab_size = 50000
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size)
vectorizer.adapt(ds_train.take(500).map(lambda x: x['title']+' '+x['description']))

> **Napomena** da koristimo samo podskup cijelog skupa podataka za izgradnju vokabulara. To radimo kako bismo ubrzali vrijeme izvršavanja i ne zadržavali vas. Međutim, preuzimamo rizik da neke riječi iz cijelog skupa podataka neće biti uključene u vokabular i bit će zanemarene tijekom treniranja. Dakle, korištenje cijele veličine vokabulara i prolazak kroz cijeli skup podataka tijekom `adapt` trebalo bi povećati konačnu točnost, ali ne značajno.

Sada možemo pristupiti stvarnom vokabularu:


In [6]:
vocab = vectorizer.get_vocabulary()
vocab_size = len(vocab)
print(vocab[:10])
print(f"Length of vocabulary: {vocab_size}")

['', '[UNK]', 'the', 'to', 'a', 'in', 'of', 'and', 'on', 'for']
Length of vocabulary: 5335


Pomoću vektorizatora možemo lako kodirati bilo koji tekst u skup brojeva:


In [7]:
vectorizer('I love to play with my words')

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 112, 3695,    3,  304,   11, 1041,    1], dtype=int64)>

## Reprezentacija teksta pomoću vreće riječi

Budući da riječi prenose značenje, ponekad možemo razumjeti značenje nekog teksta samo promatrajući pojedinačne riječi, bez obzira na njihov redoslijed u rečenici. Na primjer, pri klasifikaciji vijesti, riječi poput *vrijeme* i *snijeg* vjerojatno ukazuju na *vremensku prognozu*, dok bi riječi poput *dionice* i *dolar* bile povezane s *financijskim vijestima*.

**Vreća riječi** (BoW) je najjednostavnija tradicionalna reprezentacija vektora za razumijevanje. Svaka riječ povezana je s indeksom vektora, a element vektora sadrži broj pojavljivanja svake riječi u danom dokumentu.

![Slika koja prikazuje kako je reprezentacija vektora vreće riječi prikazana u memoriji.](../../../../../translated_images/hr/bag-of-words-example.606fc1738f1d7ba9.webp)

> **Note**: BoW možete zamisliti i kao zbroj svih vektora kodiranih metodom "jedan na jedan" za pojedinačne riječi u tekstu.

Ispod je primjer kako generirati reprezentaciju vreće riječi koristeći Python biblioteku Scikit Learn:


In [8]:
from sklearn.feature_extraction.text import CountVectorizer
sc_vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
sc_vectorizer.fit_transform(corpus)
sc_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

Također možemo koristiti Keras vektorizator koji smo definirali gore, pretvarajući svaki broj riječi u one-hot kodiranje i zbrajajući sve te vektore:


In [9]:
def to_bow(text):
    return tf.reduce_sum(tf.one_hot(vectorizer(text),vocab_size),axis=0)

to_bow('My dog likes hot dogs on a hot day.').numpy()

array([0., 5., 0., ..., 0., 0., 0.], dtype=float32)

> **Napomena**: Možda ćete biti iznenađeni što se rezultat razlikuje od prethodnog primjera. Razlog je taj što u Keras primjeru duljina vektora odgovara veličini vokabulara, koji je izgrađen iz cijelog AG News skupa podataka, dok smo u Scikit Learn primjeru vokabular izgradili iz uzorka teksta u hodu.


## Treniranje BoW klasifikatora

Sada kada smo naučili kako izraditi reprezentaciju teksta pomoću vreće riječi, idemo trenirati klasifikator koji je koristi. Prvo, trebamo naš skup podataka pretvoriti u reprezentaciju vreće riječi. To se može postići korištenjem funkcije `map` na sljedeći način:


In [11]:
batch_size = 128

ds_train_bow = ds_train.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)
ds_test_bow = ds_test.map(lambda x: (to_bow(x['title']+x['description']),x['label'])).batch(batch_size)

Sada definirajmo jednostavnu neuronsku mrežu klasifikatora koja sadrži jedan linearni sloj. Veličina ulaza je `vocab_size`, a veličina izlaza odgovara broju klasa (4). Budući da rješavamo zadatak klasifikacije, završna aktivacijska funkcija je **softmax**:


In [12]:
model = keras.models.Sequential([
    keras.layers.Dense(4,activation='softmax',input_shape=(vocab_size,))
])
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train_bow,validation_data=ds_test_bow)

938/938 [==============================] - 66s 70ms/step - loss: 0.6144 - acc: 0.8427 - val_loss: 0.4416 - val_acc: 0.8697


Budući da imamo 4 klase, točnost iznad 80% je dobar rezultat.

## Treniranje klasifikatora kao jedne mreže

Budući da je vektorizator također Keras sloj, možemo definirati mrežu koja ga uključuje i trenirati je od početka do kraja. Na taj način ne moramo vektorizirati skup podataka koristeći `map`, već možemo jednostavno proslijediti izvorni skup podataka na ulaz mreže.

> **Napomena**: I dalje bismo morali primijeniti mapiranja na naš skup podataka kako bismo polja iz rječnika (kao što su `title`, `description` i `label`) pretvorili u tuple. Međutim, prilikom učitavanja podataka s diska, možemo odmah izgraditi skup podataka s potrebnom strukturom.


In [13]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

inp = keras.Input(shape=(1,),dtype=tf.string)
x = vectorizer(inp)
x = tf.reduce_sum(tf.one_hot(x,vocab_size),axis=1)
out = keras.layers.Dense(4,activation='softmax')(x)
model = keras.models.Model(inp,out)
model.summary()

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 tf.one_hot (TFOpLambda)     (None, None, 5335)        0         
                                                                 
 tf.math.reduce_sum (TFOpLam  (None, 5335)             0         
 bda)                                                            
                                                                 
 dense_2 (Dense)             (None, 4)                 21344     
                                                                 
Total params: 21,344
Trainable params: 21,344
Non-trainable p

## Bigrami, trigrami i n-grami

Jedno ograničenje pristupa vreće riječi (bag-of-words) je to što su neke riječi dio višerječnih izraza. Na primjer, riječ 'hot dog' ima potpuno drugačije značenje od riječi 'hot' i 'dog' u drugim kontekstima. Ako uvijek predstavljamo riječi 'hot' i 'dog' koristeći iste vektore, to može zbuniti naš model.

Kako bismo to riješili, često se koriste **n-gramske reprezentacije** u metodama klasifikacije dokumenata, gdje je učestalost svake riječi, dvorječnih ili trovrječnih izraza korisna značajka za treniranje klasifikatora. U bigramskim reprezentacijama, na primjer, dodajemo sve parove riječi u vokabular, uz originalne riječi.

Ispod je primjer kako generirati bigramsku reprezentaciju vreće riječi koristeći Scikit Learn:


In [14]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

Glavni nedostatak pristupa n-grama je taj što veličina vokabulara počinje iznimno brzo rasti. U praksi, potrebno je kombinirati n-gram reprezentaciju s tehnikom smanjenja dimenzionalnosti, poput *ugradbi* (embeddings), o čemu ćemo raspravljati u sljedećoj jedinici.

Kako bismo koristili n-gram reprezentaciju u našem **AG News** skupu podataka, trebamo proslijediti parametar `ngrams` našem konstruktoru `TextVectorization`. Duljina vokabulara bigrama je **znatno veća**, u našem slučaju više od 1,3 milijuna tokena! Stoga ima smisla ograničiti bigram tokene na neki razuman broj.

Mogli bismo koristiti isti kod kao gore za treniranje klasifikatora, no to bi bilo vrlo neučinkovito u smislu memorije. U sljedećoj jedinici, trenirat ćemo klasifikator bigrama koristeći ugradbe. U međuvremenu, možete eksperimentirati s treniranjem klasifikatora bigrama u ovoj bilježnici i vidjeti možete li postići veću točnost.


## Automatsko izračunavanje BoW vektora

U gornjem primjeru izračunali smo BoW vektore ručno zbrajajući one-hot kodiranja pojedinačnih riječi. Međutim, najnovija verzija TensorFlow-a omogućuje nam automatsko izračunavanje BoW vektora prosljeđivanjem parametra `output_mode='count` konstruktoru vektorizatora. Ovo značajno olakšava definiranje i treniranje našeg modela:


In [15]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='count'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 7s 7ms/step - loss: 0.5929 - acc: 0.8486 - val_loss: 0.4168 - val_acc: 0.8772


## Frekvencija pojma - inverzna frekvencija dokumenta (TF-IDF)

U BoW reprezentaciji, pojavljivanja riječi se ponderiraju istom tehnikom bez obzira na samu riječ. Međutim, jasno je da su učestale riječi poput *a* i *in* mnogo manje važne za klasifikaciju od specijaliziranih pojmova. U većini NLP zadataka neke riječi su relevantnije od drugih.

**TF-IDF** označava **frekvenciju pojma - inverznu frekvenciju dokumenta**. To je varijacija vreće riječi, gdje se umjesto binarne vrijednosti 0/1 koja označava pojavljivanje riječi u dokumentu koristi vrijednost s pomičnim zarezom, koja je povezana s učestalošću pojavljivanja riječi u korpusu.

Formalnije, težina $w_{ij}$ riječi $i$ u dokumentu $j$ definirana je kao:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
gdje
* $tf_{ij}$ je broj pojavljivanja $i$ u $j$, tj. BoW vrijednost koju smo ranije vidjeli
* $N$ je broj dokumenata u zbirci
* $df_i$ je broj dokumenata koji sadrže riječ $i$ u cijeloj zbirci

TF-IDF vrijednost $w_{ij}$ raste proporcionalno broju puta koliko se riječ pojavljuje u dokumentu i smanjuje se ovisno o broju dokumenata u korpusu koji sadrže tu riječ, što pomaže u prilagodbi činjenici da se neke riječi pojavljuju češće od drugih. Na primjer, ako se riječ pojavljuje u *svakom* dokumentu u zbirci, $df_i=N$, i $w_{ij}=0$, te bi ti pojmovi bili potpuno zanemareni.

TF-IDF vektorizaciju teksta možete jednostavno kreirati koristeći Scikit Learn:


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

U Kerasu, sloj `TextVectorization` može automatski izračunati TF-IDF frekvencije prosljeđivanjem parametra `output_mode='tf-idf'`. Ponovimo kod koji smo koristili gore kako bismo vidjeli povećava li korištenje TF-IDF-a točnost:


In [17]:
model = keras.models.Sequential([
    keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,output_mode='tf-idf'),
    keras.layers.Dense(4,input_shape=(vocab_size,), activation='softmax')
])
print("Training vectorizer")
model.layers[0].adapt(ds_train.take(500).map(extract_text))
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 12s 12ms/step - loss: 0.4197 - acc: 0.8662 - val_loss: 0.3432 - val_acc: 0.8849


## Zaključak

Iako TF-IDF reprezentacije dodjeljuju težine frekvencijama različitih riječi, one nisu sposobne prikazati značenje ili redoslijed. Kao što je poznati lingvist J. R. Firth rekao 1935. godine: "Potpuno značenje riječi uvijek je kontekstualno, i nijedno proučavanje značenja izvan konteksta ne može se smatrati ozbiljnim." Kasnije u tečaju naučit ćemo kako uhvatiti kontekstualne informacije iz teksta koristeći jezično modeliranje.



---

**Odricanje od odgovornosti**:  
Ovaj dokument je preveden korištenjem AI usluge za prevođenje [Co-op Translator](https://github.com/Azure/co-op-translator). Iako nastojimo osigurati točnost, imajte na umu da automatski prijevodi mogu sadržavati pogreške ili netočnosti. Izvorni dokument na izvornom jeziku treba smatrati mjerodavnim izvorom. Za ključne informacije preporučuje se profesionalni prijevod od strane stručnjaka. Ne preuzimamo odgovornost za bilo kakva nesporazuma ili pogrešna tumačenja koja mogu proizaći iz korištenja ovog prijevoda.
